# QOSF Screening Tasks
I've included the solutions and my reasoning for each one. The requirement is one task completed, but I wanted to challenge myself a little, so I've done more than one.

## Task 2 - Complex Amplitudes
**Input**: A list or array of four complex amplitudes `[a0, a1, a2, a3]` that define the desired two-qubit state.
   - Ensure that the state is normalized:
     $|a_0|^2 + |a_1|^2 + |a_2|^2 + |a_3|^2 = 1$.
   - If the input is not normalized, include a normalization step.

**Output**: A representation of the two-qubit quantum state vector, for example as a NumPy array:
     $\ket{\psi} = a_0\ket{00} + a_1\ket{01} + a_2\ket{10} + a_3\ket{11}.$
 
 - Do not use quantum-specific state preparation functions from libraries.

**Testing**:
   - Write unit tests that check:
     - Normalization is enforced.
     - The output vector has the correct dimension (4 for two qubits).

**Stretch Goal**: Generalize the implementation to support a three-qubit state given 8 amplitudes.

### Solution
---
The following solution is already generalized to $n$ qubits. The number of amplitudes must be $2^n$. I think this solution is pretty self-explanatory, you just take an array of complex numbers, normalize them if needed, and check the dimensionality.

In [5]:
import numpy as np

def prepare_state(amplitudes):
    state = np.array(amplitudes, dtype=np.complex128)

    # Normalize if sum is not 1
    norm = np.linalg.norm(state)
    if not np.isclose(norm, 1.0):
        state = state / norm

    # Check dimension (must be 2^n)
    n_qubits = int(np.log2(len(state)))
    if 2**n_qubits != len(state):
        raise ValueError("Number of amplitudes must be 2^n for n qubits")
    
    return state

The unit tests are provided below.

In [ ]:
import unittest

class TestPrepareState(unittest.TestCase):
    def test_normalization(self):
        amps = [0, 1+1j, 0, 0] # (1 + 1i)|01>  -> ((1 + 1i) / sqrt(2)) |01>
        state = prepare_state(amps)
        self.assertTrue(np.isclose(np.linalg.norm(state), 1.0))
    
    def test_dimension(self):
        amps = [1, 0, 0, 0] # |00>
        state = prepare_state(amps)
        self.assertEqual(state.shape, (4,))
    
    def test_invalid_dimension(self):
        with self.assertRaises(ValueError):
            prepare_state([1, 0, 0])  # not 2^n

unittest.main(argv=[''], verbosity=2, exit=False)

#### Extended solution
Since I've tried tackling error correction in task 3, I've put together a quick and dirty library called *qlib* to make my life easier. You can check out the source in the qlib folder, and you can also find it on my github under the name [qipy](https://github.com/vladimirdabic/qipy).

The library focues mainly on density matrices rather than state vectors. It supports two types of simulation: dense and sparse. In dense simulation, it represents the density matrix as a numpy matrix, and performs direct operations on it. In sparse simulation, it represents the density matrix as a list of terms with coefficients. These terms represent entries into the density matrix.

**NOTE**: The sparse simulation stores only *non-zero* density matrix elements. For states with few non-zero terms, this can significantly reduce memory usage compared to storing the full $2^n \times 2^n$ matrix. However, as the number of terms grows, the sparse representation becomes less efficient than dense arrays.

The following examples use the dense simulation. Switching between dense and sparse is not too difficult, as all important functions and classes share the same signature. A state is initialized similarly to the solution of task 2:

In [8]:
import qlib.dense as qld
import qlib as ql

two_qubit_state = qld.State(amplitudes=[1, 0, 0, 1], latex_symbol=r"\rho_{\Phi^+}")  # \phi^+ bell state
three_qubit_state = qld.State(num_qubits=3, latex_symbol=r"\rho_{\text{three}}") # |000> state

display(two_qubit_state, three_qubit_state)

\rho_{\Phi^+} = \frac{1}{2}|00\rangle\langle 00| + \frac{1}{2}|00\rangle\langle 11| + \frac{1}{2}|11\rangle\langle 00| + \frac{1}{2}|11\rangle\langle 11|

\rho_{\text{three}} = |000\rangle\langle 000|

As you can see, a $\LaTeX$ representation is supported, which is quite nice :)

Basic circuit functionality is supported via the Circuit class:

In [30]:
import qlib.dense as qld
import qlib as ql
from IPython.display import Latex

bell_circuit = qld.Circuit(num_qubits=2)
bell_circuit.add_gate(ql.ops.H, qubit=0)
bell_circuit.CNOT(control_qubit=0, target_qubit=1)

state_00 = qld.State(amplitudes=[1, 0, 0, 0])
state_01 = qld.State(amplitudes=[0, 1, 0, 0])

final1 = bell_circuit.run(initial_state=state_00) # produces \phi^+ state
final2 = bell_circuit.run(initial_state=state_01) # produces \psi^+ state

final1.latex_symbol=r"\rho_{\Phi^+}"
final2.latex_symbol=r"\rho_{\Psi^+}"

display(Latex("$$" + state_00.to_latex() + r" \, \to \, " + final1.to_latex() + "$$"))
display(Latex("$$" + state_01.to_latex() + r" \, \to \," + final2.to_latex() + "$$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Basic noise modelling is also supported via the NoiseModel abstract class:

In [18]:
import qlib as ql
import qlib.dense as qld

# Assumes a single qubit system
class BitFlipChannel(qld.NoiseModel):
    def __init__(self, p: int):
        super().__init__()
        self.p = p

    def apply(self, state: qld.State):
        rho = state.matrix
        rho_noisy = (1 - self.p) * rho + self.p * (ql.ops.X @ rho @ ql.ops.X)
        state.matrix = rho_noisy

circuit = qld.Circuit(num_qubits=1)
circuit.add_gate(ql.ops.X, qubit=0, noise_model=BitFlipChannel(0.2))   # Noisy X gate

state = qld.State(num_qubits=1)
final = circuit.run(initial_state=state)

final

\rho = \frac{1}{5}|0\rangle\langle 0| + \frac{4}{5}|1\rangle\langle 1|

Below is an example of measuring the output of the Bell circuit with multiple shots using the sparse simulation.

In [38]:
import qlib as ql
import qlib.sparse as qls

circuit = qls.Circuit(num_qubits=2)
circuit.add_gate(ql.ops.H, qubit=0)
circuit.CNOT(control_qubit=0, target_qubit=1)

state = qls.State(num_qubits=2)
results = {}

# run_shots yields the final state for each run
for final in circuit.run_shots(initial_state=state, shots=100):
    meas_res = final.measure(qubits=[0, 1])
    res_str = ''.join(str(i) for i in meas_res)

    results[res_str] = results.get(res_str, 0) + 1

results

{'11': 55, '00': 45}

## Task 3 - Error Correction
1. Build a function to create a simple noise model. Introduce a random Pauli (X with “a” probability, Z with “b” probability) into any circuit. Test the noise model with simple circuits. 
2. Code the quantum repetition code: https://errorcorrectionzoo.org/c/quantum_repetition. Test the code with the noise model and only X errors. Why does the method not work for Z errors? 
3. Code the Shor code: https://errorcorrectionzoo.org/c/shor_nine. Test the code with your noise model.
4. Code the Hamming code: https://errorcorrectionzoo.org/c/hamming743. Test your code with your noise model. 
5. What are the differences between the Shor and Hamming codes? 
6. What challenges have you detected in the process of building the error-correcting codes? 

### Solution
---
#### 1. Simple Noise Model

Theoretically, this is called a quantum channel. A quantum channel $\mathcal{E}$ maps a state $\rho$ to another state $\rho'$. In this case, states are represented via density matrices (or density operators) which are constructed by taking the outer product of a state: $\rho = \ket{\psi}\bra{\psi}$.

The quantum channel $\mathcal{E}$ for a random Pauli X, also known as the bit-flip channel, is given by:
    $$\mathcal{E}(\rho) = (1-p) \rho + p X \rho X^\dagger, $$
     where $p$ is the probabiliy that a bit-flip occurs. Similarly, the quantum channel for a random Pauli Z, also know as the phase-flip channel, is given by:
    $$\mathcal{E}(\rho) = (1-p) \rho + p Z \rho Z^\dagger. $$
We can generalize this for any Pauli matrix $M$ and call it a Pauli channel:
    $$\mathcal{E}(\rho) = (1-p) \rho + p M \rho M^\dagger. $$

**NOTE**: I've put together a quick library called *qlib* to make my life easier. Writing it all out in this notebook makes no practical sense. Refer to Task 2 for further explanation. In this task we'll be using the dense simulation. The library comes with an abstract class called *NoiseModel*, which we should inherit and override the apply method:

In [64]:
import qlib as ql
import qlib.dense as qld
import numpy as np

class PauliNoiseModel(qld.NoiseModel):
    def __init__(self, a: int, b: int):
        super().__init__()
        self.a = a
        self.b = b
    
    def pauli_channel(self, rho: np.ndarray, p: int, pauli: np.ndarray) -> np.ndarray:
        return (1 - p) * rho + p * (pauli @ rho @ pauli.conj().T)

    def apply(self, state: qld.State):
        state.matrix = self.pauli_channel(state.matrix, self.a, ql.ops.X)
        state.matrix = self.pauli_channel(state.matrix, self.b, ql.ops.Z)

**NOTE**: The question is a little ambiguous to me, so in the case that you meant apply exactly *one* Pauli gate, then the correct channel would be:
   $$ \mathcal{E}(\rho) = (1 - a - b) \rho + a X \rho X + b Z \rho Z, $$
   where $a$ and $b$ are probabilities for a bit-flip and phase-flip, respectively. The noise model is then just:

In [65]:
class PauliNoiseModel(qld.NoiseModel):
    def __init__(self, a: int, b: int):
        super().__init__()
        
        self.a = a
        self.b = b
        self.prob_sum = a + b

        if self.prob_sum > 1 or self.prob_sum < 0:
            raise Exception("Total probability must be in the range [0, 1]")
    
    def channel(self, rho: np.ndarray) -> np.ndarray:
        return (1 - self.prob_sum) * rho + self.a * (ql.ops.X @ rho @ ql.ops.X) + self.b * (ql.ops.Z @ rho @ ql.ops.Z)

    def apply(self, state: qld.State):
        state.matrix = self.channel(state.matrix)

From this point on, I assume the sequential approach. We shall now generalize this noise model to a system of $n$ qubits. This is a quite forward step; all we have to do is introduce a qubit parameter for the channel function, and then apply this channel to each qubit independently:

In [83]:
import qlib as ql
import qlib.dense as qld
import numpy as np

class PauliNoiseModel(qld.NoiseModel):
    def __init__(self, a: int, b: int):
        super().__init__()
        self.a = a
        self.b = b
    
    def pauli_channel(self, state: qld.State, qubit: int, p: int, pauli: np.ndarray):
        gate = ql.ops.build_operator(state.num_qubits, qubit, pauli)
        state.matrix = (1 - p) * state.matrix + p * (gate @ state.matrix @ gate.conj().T)

    def apply(self, state: qld.State):
        # Apply noise to each physical qubit independently
        for qubit in range(state.num_qubits):
            self.pauli_channel(state, qubit, self.a, ql.ops.X)
            self.pauli_channel(state, qubit, self.b, ql.ops.Z)

You might have noticed the `ql.ops.build_operator(num_qubits, qubit, U)` function. This function builds an operator which is only applied to the specified qubit. If we have a 3 qubit system (dimension of $\rho$ is $2^3$), then `U_2 = ql.ops.build_operator(3, 1, U)` produces the following:
$$ U_2 = I \otimes U \otimes I $$

**NOTE**: It should be obvious that in this noise model, we assume that each qubit might *indepedently* experience a bit-flip with probability $a$ and after that a phase-flip with probability $b$. This plays an important role in the next part.

#### 2. Repetition code

Repetition coding encodes a single qubit $\ket{\psi}$ into $n$ qubits. These qubits together are called a *logical qubit* and are usually denoted by $\ket{\psi_{\mathscr{l}n}}$. The general state of a logical qubit can be described by a superposition:

$$ \ket{\psi_{\mathscr{l}n}} = \alpha \ket{0_{\mathscr{l}n}} + \beta \ket{1_{\mathscr{l}n}}. $$

We shall consider the case where $n = 3$ for simplicity. The logical basis states are then given by repeating the computational basis states 3 times:
$$ \ket{0_{\mathscr{l}3}} \to \ket{0}^{\otimes 3} = \ket{000}, $$
$$ \ket{1_{\mathscr{l}3}} \to \ket{1}^{\otimes 3} = \ket{111}. $$

Error correction is performed in two steps:
  - Eigenvalue measurements of *stabilizers* (I like to call them parity operators)
  - Flipping a qubit based on the measurement results

Stabilizers are a special kind of operator that measure the parity between adjacent qubits. In repetition coding, the set of stabilizers $\mathcal{S}$ for a $n$ logical qubit is:

$$ \mathcal{S} = \{ Z_1 Z_2, \dots, Z_{n - 1} Z_n  \}, $$

where the subscripts indicate which qubit the operator operates on. In our case of $n = 3$, the set becomes:

$$ \mathcal{S} = \{ Z Z I, I Z Z  \}, $$

where $Z Z I$ is shorthand notation for $Z \otimes Z \otimes I$, and so on. These two stabilizers have the following measurement outcomes:
  - $ZZI$ returns +1 if qubits 1 and 2 have the same value, -1 if different
  - $IZZ$ returns +1 if qubits 2 and 3 have the same value, -1 if different
  - **NOTE**: +1 and -1 are the eigenvalues of stabilizers

The following table provides the correction step based on the outcome of measuring ZZI and IZZ.
| Error     | State after error                      | ZZI | IZZ | Correction |
| --------  | -------------------------------------  | -- | -- | ------------ |
| None      | $\alpha \ket{000} + \beta \ket{111}$   | +1 | +1 | None         |
| Qubit 1   | $\alpha \ket{100} + \beta \ket{011}$   | -1 | +1 | Flip qubit 1 |
| Qubit 2   | $\alpha \ket{010} + \beta \ket{101}$   | -1 | -1 | Flip qubit 2 | 
| Qubit 3   | $\alpha \ket{001} + \beta \ket{110}$   | +1 | -1 | Flip qubit 3 |

For phase-flip (Z) errors:
   - $Z_1 \ket{\psi_{\mathscr{l}3}} = \alpha|000\rangle - \beta|111\rangle$
   - $Z_2 \ket{\psi_{\mathscr{l}3}} = \alpha|000\rangle - \beta|111\rangle$
   - $Z_3 \ket{\psi_{\mathscr{l}3}} = \alpha|000\rangle - \beta|111\rangle$

All Z errors produce the same state (differing only by a global phase), so the stabilizers cannot distinguish between them.

In [67]:
import qlib as ql
import qlib.dense as qld
import numpy as np

def repetition_code_correction(state: qld.State):
    num_qubits = state.num_qubits
    if num_qubits != 3:
        raise ValueError("Repetition code requires 3 qubits")
    
    # Stabilizers
    ZZI = ql.ops.build_operators(state.num_qubits, [0, 1], [ql.ops.Z, ql.ops.Z])
    IZZ = ql.ops.build_operators(state.num_qubits, [1, 2], [ql.ops.Z, ql.ops.Z])
    
    # Measurement
    # Expectation value gives the average value, so it's not going to be +1 or -1 exactly
    synd1 = state.expectation_value(ZZI)
    synd2 = state.expectation_value(IZZ)

    correction_applied = "None"
    if synd1 < 0 and synd2 > 0:  # (-1, +1)
        state.apply_gate(ql.ops.X, qubit=0)
        correction_applied = "X on qubit 0"
    elif synd1 < 0 and synd2 < 0:  # (-1, -1)  
        state.apply_gate(ql.ops.X, qubit=1)
        correction_applied = "X on qubit 1"
    elif synd1 > 0 and synd2 < 0:  # (+1, -1)
        state.apply_gate(ql.ops.X, qubit=2)
        correction_applied = "X on qubit 2"

    return synd1, synd2, correction_applied

The following is a *deterministic error* (noiseless) test of the correction function with a single bit flip. The initial logical state $\ket{\psi_{\mathscr{l}3}}$ is given by an equal superposition:

$$ \ket{\psi_{\mathscr{l}3}} = \frac{1}{\sqrt{2}} ( \ket{0_{\mathscr{l}3}} + \ket{1_{\mathscr{l}3}} ) = \frac{1}{\sqrt{2}} ( \ket{000} + \ket{111} ). $$

Every bit is flipped one by one, corrected and then the results are documented.

In [69]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(3):
    original_state = qld.State(amplitudes=[1, 0, 0, 0, 0, 0, 0, 1])  # 1/sqrt(2)(|0_L> + |1_L>) = 1/sqrt(2)(|000> + |111>)
    
    circuit = qld.Circuit(num_qubits=3)
    circuit.add_gate(ql.ops.X, qubit=i) # Apply a specific X error to qubit 0
    state = circuit.run(initial_state=original_state)
    
    states_error.append('\n'.join(f"P({i:03b}) = {np.abs(prob):.3f}" for i, prob in enumerate(state.probabilities()) if abs(prob) > 0.001))
    (synd1, synd2, corr) = repetition_code_correction(state)

    syndromes.append((synd1, synd2, corr))
    states_corrected.append('\n'.join(f"P({i:03b}) = {np.abs(prob):.3f}" for i, prob in enumerate(state.probabilities()) if abs(prob) > 0.001))

data = {
    "Error": ["Qubit 0", "Qubit 1", "Qubit 2"],
    "State after error": states_error,
    "ZZI": [v[0].real for v in syndromes],
    "IZZ": [v[1].real for v in syndromes],
    "Correction": [v[2] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>")) # I don't understand why pandas doesn't just allow new lines...

,Error,State after error,ZZI,IZZ,Correction,State after correction
0,Qubit 0,P(011) = 0.500P(100) = 0.500,-1.0,1.0,X on qubit 0,P(000) = 0.500P(111) = 0.500
1,Qubit 1,P(010) = 0.500P(101) = 0.500,-1.0,-1.0,X on qubit 1,P(000) = 0.500P(111) = 0.500
2,Qubit 2,P(001) = 0.500P(110) = 0.500,1.0,-1.0,X on qubit 2,P(000) = 0.500P(111) = 0.500


The following is the same test but with the noise model from step 1. The probabiliy of a bit-flip occuring for each qubit is $a = 0.8$.

In [82]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(3):
    original_state = qld.State(amplitudes=[1, 0, 0, 0, 0, 0, 0, 1])  # 1/sqrt(2)(|0_L> + |1_L>) = 1/sqrt(2)(|000> + |111>)
    
    circuit = qld.Circuit(num_qubits=3)
    circuit.add_gate(ql.ops.X, qubit=i) # Apply a specific X error to qubit 0
    state = circuit.run(initial_state=original_state, noise_model=PauliNoiseModel(0.8, 0))
    temp_for_table = []
    
    for i, prob in enumerate(state.matrix.diagonal()):
        if abs(prob) > 0.001:
            temp_for_table.append(f"P({i:03b}) = {np.abs(prob):.3f}")

    states_error.append('\n'.join(temp_for_table))
    (synd1, synd2, corr) = repetition_code_correction(state)

    syndromes.append((synd1, synd2, corr))

    temp_for_table.clear()
    for i, prob in enumerate(state.matrix.diagonal()):
        if abs(prob) > 0.001:
            temp_for_table.append(f"P({i:03b}) = {np.abs(prob):.3f}")

    states_corrected.append('\n'.join(temp_for_table))

data = {
    "Error": ["Qubit 0", "Qubit 1", "Qubit 2"],
    "State after error": states_error,
    "ZZI": [v[0].real for v in syndromes],
    "IZZ": [v[1].real for v in syndromes],
    "Correction": [v[2] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>"))

,Error,State after error,ZZI,IZZ,Correction,State after correction
0,Qubit 0,P(000) = 0.080P(001) = 0.080P(010) = 0.080P(011) = 0.260P(100) = 0.260P(101) = 0.080P(110) = 0.080P(111) = 0.080,-0.36,0.36,X on qubit 0,P(000) = 0.260P(001) = 0.080P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.080P(111) = 0.260
1,Qubit 1,P(000) = 0.080P(001) = 0.080P(010) = 0.260P(011) = 0.080P(100) = 0.080P(101) = 0.260P(110) = 0.080P(111) = 0.080,-0.36,-0.36,X on qubit 1,P(000) = 0.260P(001) = 0.080P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.080P(111) = 0.260
2,Qubit 2,P(000) = 0.080P(001) = 0.260P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.260P(111) = 0.080,0.36,-0.36,X on qubit 2,P(000) = 0.260P(001) = 0.080P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.080P(111) = 0.260


We can see that after correction, the states $\ket{000} = \ket{0_{\mathscr{l}3}}$ and $\ket{111} = \ket{1_{\mathscr{l}3}}$ stand out from the rest. The rest have probabilty of being measured due to:

- Residual uncorrected errors
- Multiple error combinations

This happens because we introduced noise into the system, but the trend is clearly toward correction.

Let's discuss practical implementation for a moment. To create the logical qubit state, we would use two CNOT gates to entangle two additional repeated qubits. Then, any unitary we want to perform on the logical qubit, we have to perform for all of the qubits: $U^{\otimes 3} = U \otimes U \otimes U$. In the circuit shown, $U$ represents a circuit operating on the logical qubit and possibly some noise.

<center><img src="img/repeatingcodeimpl.png"/></center>

After the circuit $U$ has been performed, we peform error correction. We introduce two ancilla qubits, which are used for stabilizer measurements. The first pair of CNOTs represent the stabilizer $ZZI$ and the second pair $IZZ$. We then measure these two ancilla qubits, and we pass the results (+1 being classical bit 0, and -1 being classical bit 1) to a field programmable gate array or microcontroller which then decides which qubits to flip.

#### 3. Shor code
Shor coding is similar to repetition coding. It is repetition coding, but applied two times. To see how, we shall first explore—in short—repetition coding that protects against phase-flips.

Repetition coding for phase-flips is achieved by repeating the Hadamard (or X) basis states $n$ times. We shall focus on $n = 3$ once again. The general state of a logical qubit $\ket{\psi_{\mathscr{l}3}}$ is, again, given by
$$\ket{\psi_{\mathscr{l}3}} = \alpha \ket{0_{\mathscr{l}3}} + \beta \ket{1_{\mathscr{l}3}}, $$
where the basis states $\ket{0_L}$ and $\ket{1_L}$ are now defined as
$$ \ket{0_{\mathscr{l}3}} = \ket{+}^{\otimes 3} = \ket{+++}, $$
$$ \ket{1_{\mathscr{l}3}} = \ket{-}^{\otimes 3} = \ket{---}. $$

Error correction is also performed via stabilizers. The stabilizers we use in this case are:
$$ \mathcal{S} = \{ XXI, IXX \}. $$

Why? Because $\ket{+}$ and $\ket{-}$ are the eigenstates of $X$. How does this protect against phase-flips exactly? A phase-flip in the computational (or Z) basis has the effect of a bit-flip in the Hadamard basis!


$$ \ket{+} = \frac{1}{\sqrt{2}} ( \ket{0} + \ket{1} ) \quad \xrightarrow{\text{phase-flip}} \quad \ket{-} = \frac{1}{\sqrt{2}} ( \ket{0} - \ket{1} ). $$


The following table provides the correction step based on the outcome of measuring XXI and IXX.
| Error     | State after error                      | XXI | IXX| Correction   |
| --------  | -------------------------------------  | -- | -- | -------------- |
| None      | $\alpha \ket{+++} + \beta \ket{---}$   | +1 | +1 | None           |
| Qubit 1   | $\alpha \ket{-++} + \beta \ket{+--}$   | -1 | +1 | $Z$ on qubit 1 |
| Qubit 2   | $\alpha \ket{+-+} + \beta \ket{-+-}$   | -1 | -1 | $Z$ on qubit 2 | 
| Qubit 3   | $\alpha \ket{++-} + \beta \ket{--+}$   | +1 | -1 | $Z$ on qubit 3 |

**NOTE**: It should be familiar to the reader that this is the same as repetition coding for bit-flips but in another basis, hence we won't be writing code to perform the same experiment (I just think its tedious).

Now, how do we combine these two repetition codes to achieve shor coding? Here's how:
   - Encode the logical states $\ket{0_{\mathscr{l}3}}$ and $\ket{1_{\mathscr{l}3}}$ with repetition coding for phase-flips
   - Encode the states $\ket{+}$ and $\ket{-}$ via repetition coding for bit-flips

Let's go backwards to make it more understandable. Encoding $\ket{+}$ and $\ket{-}$ via repetition coding for bit-flips gives us the following states
$$ \ket{+_{\mathscr{l}3}} = \frac{1}{\sqrt{2}} (\ket{0_{\mathscr{l}3}} + \ket{1_{\mathscr{l}3}}), \quad \ket{-_{\mathscr{l}3}} = \frac{1}{\sqrt{2}} (\ket{0_{\mathscr{l}3}} - \ket{1_{\mathscr{l}3}}). $$

We now take these states, encode a logical qubit with repetition coding for phase-flips, substitute, and obtain: 
$$ \ket{0_{\mathscr{l}9}} = \ket{+_{\mathscr{l}3}}^{\otimes 3} = \frac{1}{2 \sqrt{2}} (\ket{000} + \ket{111}) (\ket{000} + \ket{111}) (\ket{000} + \ket{111}), $$
$$ \ket{1_{\mathscr{l}9}} = \ket{-_{\mathscr{l}3}}^{\otimes 3} = \frac{1}{2 \sqrt{2}} (\ket{000} - \ket{111}) (\ket{000} - \ket{111}) (\ket{000} - \ket{111}). $$

In total, we need 9 physical qubits to represent a logical one. We note that there are three groups of three physical qubits.

Error correction is performed via two sets of stabilizers: the set of stabilizers $\mathcal{S}_b$ for a bit-flip and $\mathcal{S}_p$ for a phase-flip:
$$ \mathcal{S}_b = \{ZZI, IZZ\}, $$
$$ \mathcal{S}_p = \{XXI, IXX\}. $$

Since we have three groups of three qubits, we want the phase-flip stabilizers to act on all three qubits, hence the stabilizers are extended to be:
$$ \mathcal{S}_b = \{ZZI, IZZ\}, $$
$$ \mathcal{S}_p = \{XXXXXXIII, IIIXXXXXX\}, $$

where—to remind the reader—$ZZI$ is shorthand notation for $Z \otimes Z \otimes I$, and so on.

**NOTE**: This is a simplification. The full set of stabilizers $\mathcal{S}$ would be
$$ ZZIIIIIII $$
$$ IZZIIIIII $$
$$ IIIZZIIII $$
$$ IIIIZZIII $$
$$ IIIIIIZZI $$
$$ IIIIIIIZZ $$
$$ XXXXXXIII $$
$$ IIIXXXXXX, $$
but I feel that this complicates things in terms of discussion for no reason.

Bit-flip corrections are performed by measuring stabilizers $\mathcal{S}_b$ within each group separately and then flipping the qubit determined by the measurement outcomes. On the other hand, phase-flip corrections are performed by measuring stabilizers $\mathcal{S}_p$ on *whole* groups and then flipping the phase of *one* qubit in the group in which the error was indentified.

The following table provides the correction step based on the outcome of measuring $\mathcal{S}_p$.
| Error     | XXXXXXIII | IIIXXXXXX | Correction     |
| --------  | ----------| --------- | -------------- |
| None      |     +1    |     +1    | None           |
| Group 1   |     -1    |     +1    | $Z$ on any qubit in group 1 |
| Group 2   |     -1    |     -1    | $Z$ on any qubit in group 2 | 
| Group 3   |     +1    |     -1    | $Z$ on any qubit in group 3 |

Below is the implementation of shor code correction.

In [94]:
import qlib as ql
import qlib.dense as qld
import numpy as np

def shor_code_correction(state: qld.State):
    num_qubits = state.num_qubits
    if num_qubits != 9:
        raise ValueError("Repetition code requires 9 qubits")
    
    # Bit-flip stabilizers (per group)
    ZZI_1 = ql.ops.build_operators(state.num_qubits, [0, 1], [ql.ops.Z, ql.ops.Z])
    IZZ_1 = ql.ops.build_operators(state.num_qubits, [1, 2], [ql.ops.Z, ql.ops.Z])
    ZZI_2 = ql.ops.build_operators(state.num_qubits, [3, 4], [ql.ops.Z, ql.ops.Z])
    IZZ_2 = ql.ops.build_operators(state.num_qubits, [4, 5], [ql.ops.Z, ql.ops.Z])
    ZZI_3 = ql.ops.build_operators(state.num_qubits, [6, 7], [ql.ops.Z, ql.ops.Z])
    IZZ_3 = ql.ops.build_operators(state.num_qubits, [7, 8], [ql.ops.Z, ql.ops.Z])

    # Phase-flip stabilizers
    XXXXXXIII = ql.ops.build_operators(state.num_qubits, list(range(9)), [ql.ops.X] * 6 + [ql.ops.I] * 3)
    IIIXXXXXX = ql.ops.build_operators(state.num_qubits, list(range(9)), [ql.ops.I] * 3 + [ql.ops.X] * 6)
    
    # Bit-flip measurements (per group)
    synd1_1 = state.expectation_value(ZZI_1)
    synd2_1 = state.expectation_value(IZZ_1)
    synd1_2 = state.expectation_value(ZZI_2)
    synd2_2 = state.expectation_value(IZZ_2)
    synd1_3 = state.expectation_value(ZZI_3)
    synd2_3 = state.expectation_value(IZZ_3)

    # Phase-flip measurements
    synd1 = state.expectation_value(XXXXXXIII)
    synd2 = state.expectation_value(IIIXXXXXX)

    corrections_applied = []
    
    # Bit-flip corrections
    if correction := correct_group(state, 0, synd1_1, synd2_1):
        corrections_applied.append(correction)

    if correction := correct_group(state, 1, synd1_2, synd2_2):
        corrections_applied.append(correction)

    if correction := correct_group(state, 2, synd1_3, synd2_3):
        corrections_applied.append(correction)
    
    # Phase-flip corrections
    if synd1 < 0 and synd2 > 0:  # (-1, +1)
        state.apply_gate(ql.ops.Z, qubit=0)
        corrections_applied.append("Z (G0)")
    elif synd1 < 0 and synd2 < 0:  # (-1, -1)  
        state.apply_gate(ql.ops.Z, qubit=3)
        corrections_applied.append("Z (G1)")
    elif synd1 > 0 and synd2 < 0:  # (+1, -1)
        state.apply_gate(ql.ops.Z, qubit=6)
        corrections_applied.append("Z (G2)")

    return synd1, synd2, ((synd1_1, synd2_1), (synd1_2, synd2_2), (synd1_3, synd2_3)), " & ".join(corrections_applied)

def correct_group(state: qld.State, group: int, synd1: np.complex128, synd2: np.complex128):
    correction_applied = None
    
    if synd1 < 0 and synd2 > 0:  # (-1, +1)
        state.apply_gate(ql.ops.X, qubit=(group * 3))
        correction_applied = f"X (Q0, G{group})"
    elif synd1 < 0 and synd2 < 0:  # (-1, -1)  
        state.apply_gate(ql.ops.X, qubit=(group * 3 + 1))
        correction_applied = f"X (Q1, G{group})"
    elif synd1 > 0 and synd2 < 0:  # (+1, -1)
        state.apply_gate(ql.ops.X, qubit=(group * 3 + 2))
        correction_applied = f"X (Q2, G{group})"

    return correction_applied

Earlier we have seen the logical states $\ket{0_{\mathscr{l}9}}$ and $\ket{1_{\mathscr{l}9}}$ for Shor encoding:
$$ \ket{0_{\mathscr{l}9}} = \frac{1}{2 \sqrt{2}} (\ket{000} + \ket{111}) (\ket{000} + \ket{111}) (\ket{000} + \ket{111}), $$
$$ \ket{1_{\mathscr{l}9}} = \frac{1}{2 \sqrt{2}} (\ket{000} - \ket{111}) (\ket{000} - \ket{111}) (\ket{000} - \ket{111}). $$

To construct these states in code, we must distribute the implied tensor product, which gives us:

$$ \ket{0_{\mathscr{l}9}} = \frac{1}{2 \sqrt{2}} (\ket{000}\ket{000}\ket{000} + \ket{000}\ket{000}\ket{111} + \ket{000}\ket{111}\ket{000} + \dots ), $$
$$ \ket{1_{\mathscr{l}9}} = \frac{1}{2 \sqrt{2}} (\ket{000}\ket{000}\ket{000} - \ket{000}\ket{000}\ket{111} - \ket{000}\ket{111}\ket{000} + \dots ). $$

The rest is easy to do in your head (note the minus signs). We can construct these logical states in code the following way:

In [86]:
def shor_logical_zero() -> qld.State:
    amps = [0] * 512
    
    # |0_l9> = (|000>+|111>)(|000>+|111>)(|000>+|111>) / (2√2)
    # Superposition of 8 basis states
    
    amps[int('000000000', 2)] = 1
    amps[int('000000111', 2)] = 1
    amps[int('000111000', 2)] = 1
    amps[int('000111111', 2)] = 1
    
    amps[int('111000000', 2)] = 1
    amps[int('111000111', 2)] = 1
    amps[int('111111000', 2)] = 1
    amps[int('111111111', 2)] = 1

    return qld.State(amplitudes=amps)

def shor_logical_one() -> qld.State:
    amps = [0] * 512
    
    # |1_l9> = (|000>-|111>)(|000>-|111>)(|000>-|111>) / (2√2)
    # Same states as |0_L> but with alternating signs
    
    amps[int('000000000', 2)] = 1
    amps[int('000000111', 2)] = -1
    amps[int('000111000', 2)] = -1
    amps[int('000111111', 2)] = 1
    
    amps[int('111000000', 2)] = -1
    amps[int('111000111', 2)] = 1
    amps[int('111111000', 2)] = 1
    amps[int('111111111', 2)] = -1
    
    return qld.State(amplitudes=amps)

We define a helper function for running tests so we don't repeat a lot of code:

In [131]:
def run_shor_test(circuit: qld.Circuit):
    state_0 = shor_logical_zero()
    state_1 = shor_logical_one()
        
    # Create superposition: |0_l9> + |1_l9>
    amps_superpos = state_0.matrix.diagonal() + state_1.matrix.diagonal()
    original_state = qld.State(amplitudes=amps_superpos)
        
    state = circuit.run(initial_state=original_state)
    temp_for_table = []
        
    for i, prob in enumerate(state.probabilities()):
        if prob > 0.001:
            temp_for_table.append(f"P({i:09b}) = {np.abs(prob):.3f}")
    
    states_error = '\n'.join(temp_for_table)
    (synd1, synd2, block_synds, corr) = shor_code_correction(state)
    
    syndromes = (synd1, synd2, block_synds, corr)
    
    temp_for_table.clear()
    for i, prob in enumerate(state.probabilities()):
        if prob > 0.001:
            temp_for_table.append(f"P({i:09b}) = {np.abs(prob):.3f}")
    
    states_corrected = '\n'.join(temp_for_table)

    return syndromes, states_error, states_corrected

The following is a *deterministic error* (noiseless) test of the correction function with a single bit flip. The initial logical state $\ket{\psi_{\mathscr{l}9}}$ is given by an equal superposition:

$$ \ket{\psi_{\mathscr{l}9}} = \frac{1}{\sqrt{2}} ( \ket{0_{\mathscr{l}9}} + \ket{1_{\mathscr{l}9}} ). $$

Every bit is flipped one by one, corrected and then the results are documented.

In [92]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(9):
    circuit = qld.Circuit(num_qubits=9)
    circuit.add_gate(ql.ops.X, qubit=i) # Apply a specific X error to qubit i

    (synd, errorstate, correctstate) = run_shor_test(circuit)
    states_error.append(errorstate)
    states_corrected.append(correctstate)
    syndromes.append(synd)

data = {
    "Error": [f"Qubit {i}" for i in range(9)],
    "State after error": states_error,
    "XXXXXXIII": [f"{v[0].real:.2f}" for v in syndromes],
    "IIIXXXXXX": [f"{v[1].real:.2f}" for v in syndromes],
    "Group 0 (ZZI, IZZ)": [f"{v[2][0][0].real:.2f}, {v[2][0][1].real:.2f}" for v in syndromes],  # this is a bit ugly, but it does the job
    "Group 1 (ZZI, IZZ)": [f"{v[2][1][0].real:.2f}, {v[2][1][1].real:.2f}" for v in syndromes],
    "Group 2 (ZZI, IZZ)": [f"{v[2][2][0].real:.2f}, {v[2][2][1].real:.2f}" for v in syndromes],
    "Correction": [v[3] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>"))

,Error,State after error,XXXXXXIII,IIIXXXXXX,"Group 0 (ZZI, IZZ)","Group 1 (ZZI, IZZ)","Group 2 (ZZI, IZZ)",Correction,State after correction
0,Qubit 0,P(011000000) = 0.125P(011000111) = 0.125P(011111000) = 0.125P(011111111) = 0.125P(100000000) = 0.125P(100000111) = 0.125P(100111000) = 0.125P(100111111) = 0.125,1.00,1.00,"-1.00, 1.00","1.00, 1.00","1.00, 1.00","X (Q0, G0)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
1,Qubit 1,P(010000000) = 0.125P(010000111) = 0.125P(010111000) = 0.125P(010111111) = 0.125P(101000000) = 0.125P(101000111) = 0.125P(101111000) = 0.125P(101111111) = 0.125,1.00,1.00,"-1.00, -1.00","1.00, 1.00","1.00, 1.00","X (Q1, G0)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
2,Qubit 2,P(001000000) = 0.125P(001000111) = 0.125P(001111000) = 0.125P(001111111) = 0.125P(110000000) = 0.125P(110000111) = 0.125P(110111000) = 0.125P(110111111) = 0.125,1.00,1.00,"1.00, -1.00","1.00, 1.00","1.00, 1.00","X (Q2, G0)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
3,Qubit 3,P(000011000) = 0.125P(000011111) = 0.125P(000100000) = 0.125P(000100111) = 0.125P(111011000) = 0.125P(111011111) = 0.125P(111100000) = 0.125P(111100111) = 0.125,1.00,1.00,"1.00, 1.00","-1.00, 1.00","1.00, 1.00","X (Q0, G1)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
4,Qubit 4,P(000010000) = 0.125P(000010111) = 0.125P(000101000) = 0.125P(000101111) = 0.125P(111010000) = 0.125P(111010111) = 0.125P(111101000) = 0.125P(111101111) = 0.125,1.00,1.00,"1.00, 1.00","-1.00, -1.00","1.00, 1.00","X (Q1, G1)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
5,Qubit 5,P(000001000) = 0.125P(000001111) = 0.125P(000110000) = 0.125P(000110111) = 0.125P(111001000) = 0.125P(111001111) = 0.125P(111110000) = 0.125P(111110111) = 0.125,1.00,1.00,"1.00, 1.00","1.00, -1.00","1.00, 1.00","X (Q2, G1)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
6,Qubit 6,P(000000011) = 0.125P(000000100) = 0.125P(000111011) = 0.125P(000111100) = 0.125P(111000011) = 0.125P(111000100) = 0.125P(111111011) = 0.125P(111111100) = 0.125,1.00,1.00,"1.00, 1.00","1.00, 1.00","-1.00, 1.00","X (Q0, G2)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
7,Qubit 7,P(000000010) = 0.125P(000000101) = 0.125P(000111010) = 0.125P(000111101) = 0.125P(111000010) = 0.125P(111000101) = 0.125P(111111010) = 0.125P(111111101) = 0.125,1.00,1.00,"1.00, 1.00","1.00, 1.00","-1.00, -1.00","X (Q1, G2)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
8,Qubit 8,P(000000001) = 0.125P(000000110) = 0.125P(000111001) = 0.125P(000111110) = 0.125P(111000001) = 0.125P(111000110) = 0.125P(111111001) = 0.125P(111111110) = 0.125,1.00,1.00,"1.00, 1.00","1.00, 1.00","1.00, -1.00","X (Q2, G2)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125


The following is a *deterministic error* (noiseless) test of the correction function with a single phase flip. The initial logical state $\ket{\psi_{\mathscr{l}9}}$ is given by an equal superposition:

$$ \ket{\psi_{\mathscr{l}9}} = \frac{1}{\sqrt{2}} ( \ket{0_{\mathscr{l}9}} + \ket{1_{\mathscr{l}9}} ). $$

Every bits' phase is flipped one by one, corrected and then the results are documented.

**NOTE**: The phase changes are not visible in the table because I show only probabilities. You can see that the error has been correctly identified, and the correction applied, in the eigenvalue and correction columns.

In [95]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(9):
    circuit = qld.Circuit(num_qubits=9)
    circuit.add_gate(ql.ops.Z, qubit=i) # Apply a specific Z error to qubit i

    (synd, errorstate, correctstate) = run_shor_test(circuit)
    states_error.append(errorstate)
    states_corrected.append(correctstate)
    syndromes.append(synd)

data = {
    "Error": [f"Qubit {i}" for i in range(9)],
    "State after error": states_error,
    "XXXXXXIII": [f"{v[0].real:.2f}" for v in syndromes],
    "IIIXXXXXX": [f"{v[1].real:.2f}" for v in syndromes],
    "Group 0 (ZZI, IZZ)": [f"{v[2][0][0].real:.2f}, {v[2][0][1].real:.2f}" for v in syndromes],  # this is a bit ugly, but it does the job
    "Group 1 (ZZI, IZZ)": [f"{v[2][1][0].real:.2f}, {v[2][1][1].real:.2f}" for v in syndromes],
    "Group 2 (ZZI, IZZ)": [f"{v[2][2][0].real:.2f}, {v[2][2][1].real:.2f}" for v in syndromes],
    "Correction": [v[3] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>"))

,Error,State after error,XXXXXXIII,IIIXXXXXX,"Group 0 (ZZI, IZZ)","Group 1 (ZZI, IZZ)","Group 2 (ZZI, IZZ)",Correction,State after correction
0,Qubit 0,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,-1.00,1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G0),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
1,Qubit 1,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,-1.00,1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G0),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
2,Qubit 2,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,-1.00,1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G0),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
3,Qubit 3,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,-1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G1),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
4,Qubit 4,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,-1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G1),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
5,Qubit 5,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,-1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G1),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
6,Qubit 6,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G2),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
7,Qubit 7,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G2),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
8,Qubit 8,P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125,1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, 1.00",Z (G2),P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125


The following is a combination of the previous tests, where a qubit experiences a bit-flip and a phase-flip at the same time.

In [56]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(9):
    circuit = ql.Circuit(num_qubits=9)
    circuit.add_gate(ql.ops.X, qubit=i)
    circuit.add_gate(ql.ops.Z, qubit=i)

    (synd, errorstate, correctstate) = run_shor_test(circuit)
    states_error.append(errorstate)
    states_corrected.append(correctstate)
    syndromes.append(synd)

data = {
    "Error": [f"Qubit {i}" for i in range(9)],
    "State after error": states_error,
    "XXXXXXIII": [f"{v[0].real:.2f}" for v in syndromes],
    "IIIXXXXXX": [f"{v[1].real:.2f}" for v in syndromes],
    "Group 0 (ZZI, IZZ)": [f"{v[2][0][0].real:.2f}, {v[2][0][1].real:.2f}" for v in syndromes],  # this is a bit ugly, but it does the job
    "Group 1 (ZZI, IZZ)": [f"{v[2][1][0].real:.2f}, {v[2][1][1].real:.2f}" for v in syndromes],
    "Group 2 (ZZI, IZZ)": [f"{v[2][2][0].real:.2f}, {v[2][2][1].real:.2f}" for v in syndromes],
    "Correction": [v[3] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>"))

,Error,State after error,XXXXXXIII,IIIXXXXXX,"Group 0 (ZZI, IZZ)","Group 1 (ZZI, IZZ)","Group 2 (ZZI, IZZ)",Correction,State after correction
0,Qubit 0,P(011000000) = 0.125P(011000111) = 0.125P(011111000) = 0.125P(011111111) = 0.125P(100000000) = 0.125P(100000111) = 0.125P(100111000) = 0.125P(100111111) = 0.125,-1.00,1.00,"-1.00, 1.00","1.00, 1.00","1.00, 1.00","X (Q0, G0) & Z (G0)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
1,Qubit 1,P(010000000) = 0.125P(010000111) = 0.125P(010111000) = 0.125P(010111111) = 0.125P(101000000) = 0.125P(101000111) = 0.125P(101111000) = 0.125P(101111111) = 0.125,-1.00,1.00,"-1.00, -1.00","1.00, 1.00","1.00, 1.00","X (Q1, G0) & Z (G0)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
2,Qubit 2,P(001000000) = 0.125P(001000111) = 0.125P(001111000) = 0.125P(001111111) = 0.125P(110000000) = 0.125P(110000111) = 0.125P(110111000) = 0.125P(110111111) = 0.125,-1.00,1.00,"1.00, -1.00","1.00, 1.00","1.00, 1.00","X (Q2, G0) & Z (G0)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
3,Qubit 3,P(000011000) = 0.125P(000011111) = 0.125P(000100000) = 0.125P(000100111) = 0.125P(111011000) = 0.125P(111011111) = 0.125P(111100000) = 0.125P(111100111) = 0.125,-1.00,-1.00,"1.00, 1.00","-1.00, 1.00","1.00, 1.00","X (Q0, G1) & Z (G1)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
4,Qubit 4,P(000010000) = 0.125P(000010111) = 0.125P(000101000) = 0.125P(000101111) = 0.125P(111010000) = 0.125P(111010111) = 0.125P(111101000) = 0.125P(111101111) = 0.125,-1.00,-1.00,"1.00, 1.00","-1.00, -1.00","1.00, 1.00","X (Q1, G1) & Z (G1)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
5,Qubit 5,P(000001000) = 0.125P(000001111) = 0.125P(000110000) = 0.125P(000110111) = 0.125P(111001000) = 0.125P(111001111) = 0.125P(111110000) = 0.125P(111110111) = 0.125,-1.00,-1.00,"1.00, 1.00","1.00, -1.00","1.00, 1.00","X (Q2, G1) & Z (G1)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
6,Qubit 6,P(000000011) = 0.125P(000000100) = 0.125P(000111011) = 0.125P(000111100) = 0.125P(111000011) = 0.125P(111000100) = 0.125P(111111011) = 0.125P(111111100) = 0.125,1.00,-1.00,"1.00, 1.00","1.00, 1.00","-1.00, 1.00","X (Q0, G2) & Z (G2)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
7,Qubit 7,P(000000010) = 0.125P(000000101) = 0.125P(000111010) = 0.125P(000111101) = 0.125P(111000010) = 0.125P(111000101) = 0.125P(111111010) = 0.125P(111111101) = 0.125,1.00,-1.00,"1.00, 1.00","1.00, 1.00","-1.00, -1.00","X (Q1, G2) & Z (G2)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125
8,Qubit 8,P(000000001) = 0.125P(000000110) = 0.125P(000111001) = 0.125P(000111110) = 0.125P(111000001) = 0.125P(111000110) = 0.125P(111111001) = 0.125P(111111110) = 0.125,1.00,-1.00,"1.00, 1.00","1.00, 1.00","1.00, -1.00","X (Q2, G2) & Z (G2)",P(000000000) = 0.125P(000000111) = 0.125P(000111000) = 0.125P(000111111) = 0.125P(111000000) = 0.125P(111000111) = 0.125P(111111000) = 0.125P(111111111) = 0.125


Applying the noise model from earlier yields similar results to ones for repetition coding. The noise is present in the system, but the corrected states stand out from the rest. For the sake of readability, I've decided to drop the cell which did this, because it produces a really huge table.  

#### 4. Hamming code

Hamming code, or Steane code (the quantum version), encodes 4 bits into 7 bits. Let's briefly discuss the classical Hamming code first.


The generator matrix $G$ is used to encode a 4-bit row vector into a 7-bit row vector. It's defined as
$$ G = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 1 & 1 \\
0 & 1 & 0 & 0 & 1 & 0 & 1 \\
0 & 0 & 1 & 0 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 \\
\end{bmatrix}. $$

To encode a 4-bit row vector $\mathbf{d}$, we multiply it by the generator matrix
$$ \mathbf{d}' = \mathbf{d} G, $$
which gives us the encoded 7-bit row vector $\mathbf{d}'$.

To recover the original 4-bit vector, we use the parity matrix $H$ to compute the syndrome $\mathbf{s}$. The matrix is defined as
$$ H = \begin{bmatrix}
1 & 0 & 1 & 0 & 1 & 0 & 1 \\
0 & 1 & 1 & 0 & 0 & 1 & 1 \\
0 & 0 & 0 & 1 & 1 & 1 & 1 \\
\end{bmatrix}, $$
which—as you might have noticed—is just counting up in binary, where the LSB is the top one. To compute the syndrome $\mathbf{s}$, we multiply the matrix $H$ with the transpose of our encoded 7-bit vector
$$ \mathbf{s} = H \mathbf{d}^{\prime T}. $$

The syndrome $\mathbf{s}$ is a 3-bit vector which tells us in what position the error has occurred.

Let's now apply this to a simple example. We want to encode $1011$, so we let $\mathbf{d} = [1, 0, 1, 1]$. We then compute $\mathbf{d}'$ and find
$$ \mathbf{d}' = \mathbf{d} G = [1, 0, 1, 1, 0, 1, 0]. $$

So, our encoded 7-bit string is $1011010$. Suppose that an error occurs, and that the 4th bit is flipped: $1010010$. $\mathbf{d}'$ then becomes
$$ \mathbf{d}' = [1, 0, 1, 0, 0, 1, 0]. $$

We now compute the syndrome $\mathbf{s}$
$$ \mathbf{s} = H \mathbf{d}^{\prime T} = [0, 0, 1], $$
and we read it off right to left, which gives us $100_2$, which in decimal is $4_{10}$. So we conclude that the 4th bit flipped.

Let's now discuss *codewords*. Codewords are data patterns that represent error-free encoded states, or in other words, states whose syndrome is $\mathbf{s} = \mathbf{0}$. Finding the codewords is quite simple. For every number from $0$ to $2^n$ in binary representation, we apply the generator matrix. Since we're working with 4-bit data, $n = 4$, that means we must apply $G$ to $0000, 0001, 0010, \dots, 1111$. Below is the table of codewords we obtain.

| 4-bit data | 7-bit encoded data | 4-bit data | 7-bit encoded data |
| ---------- | ------------------ | ---------- | ------------------ |
| 0000       | 0000000            | 1000       | 1000011            |
| 0001       | 0001111            | 1001       | 1001100            |
| 0010       | 0010110            | 1010       | 1010101            |
| 0011       | 0011001            | 1011       | 1011010            |
| 0100       | 0100101            | 1100       | 1100110            |
| 0101       | 0101010            | 1101       | 1101001            |
| 0110       | 0110011            | 1110       | 1110000            |
| 0111       | 0111100            | 1111       | 1111111            |

These codewords play a key role in the quantum version of the Hamming code, called the *Steane code*. They are used to form the logical basis states $\ket{0_{\mathscr{l}7}}$ and $\ket{1_{\mathscr{l}7}}$.

The logical state $\ket{0_{\mathscr{l}7}}$ is defined as an equal superposition of all *even* weight (contains an even number of 1's) codewords, and the logical state $\ket{1_{\mathscr{l}7}}$ is also defined as an equal superposition but of all *odd* (contains an odd number of 1's) weight codewords. They are

$$ \begin{align*}
\ket{0_{\mathscr{l}7}} = (2\sqrt{2})^{-1} (&\ket{0000000} + \ket{0001111} + \ket{0110011} \\
                                             + &\ket{0111100} + \ket{1011010} + \ket{1100110} \\
                                             + &\ket{1101001} + \ket{1010101}),
\end{align*}
$$

$$ \begin{align*}
\ket{1_{\mathscr{l}7}} = (2\sqrt{2})^{-1} (&\ket{1111111} + \ket{1110000} + \ket{1001100} \\
                                             + &\ket{1000011} + \ket{0100101} + \ket{0011001} \\
                                             + &\ket{0010110} + \ket{0101010}).
\end{align*}
$$

The stabilizers are defined with the help of the parity matrix $H$ from earlier. Other than it counting up in binary, the parity matrix has another interesting property, it defines the parity equations. If an encoded state satisfies the parity equations, it is considered a codeword. If we read off each row from $H$, we find the following equations:
   - Row 1: $1010101 \to d_1 \oplus d_3 \oplus d_5 \oplus d_7 = 0$
   - Row 2: $0110011 \to d_2 \oplus d_3 \oplus d_6 \oplus d_7 = 0$
   - Row 3: $0001111 \to d_4 \oplus d_5 \oplus d_6 \oplus d_7 = 0$

Stabilizers are formed by promoting these equations to operators. For bit-flip errors, we promote them to $Z$ operators
$$ \begin{align*}
d_1 \oplus d_3 \oplus d_5 \oplus d_7 = 0 \quad &\to \quad Z_1 Z_3 Z_5 Z_7 \\
d_2 \oplus d_3 \oplus d_6 \oplus d_7 = 0 \quad &\to \quad Z_2 Z_3 Z_6 Z_7 \\
d_4 \oplus d_5 \oplus d_6 \oplus d_7 = 0 \quad &\to \quad Z_4 Z_5 Z_6 Z_7,
\end{align*}
$$
and for phase-flip errors to $X$ operators
$$ \begin{align*}
d_1 \oplus d_3 \oplus d_5 \oplus d_7 = 0 \quad &\to \quad X_1 X_3 X_5 X_7 \\
d_2 \oplus d_3 \oplus d_6 \oplus d_7 = 0 \quad &\to \quad X_2 X_3 X_6 X_7 \\
d_4 \oplus d_5 \oplus d_6 \oplus d_7 = 0 \quad &\to \quad X_4 X_5 X_6 X_7.
\end{align*}
$$

Errors are found by comparing the measurement results of these stabilizers. The table for bit-flip errors is shown below.

| $Z_1 Z_3 Z_5 Z_7$ | $Z_2 Z_3 Z_6 Z_7$ | $Z_4 Z_5 Z_6 Z_7$ | Binary syndrome | Correction   |
| ----------------- | ----------------- | ----------------- | --------------  | ---------    |
|        +1         |         +1        |         +1        |   000           | None         |
|        -1         |         +1        |         +1        |   001           | X on qubit 1 |
|        +1         |         -1        |         +1        |   010           | X on qubit 2 |
|        -1         |         -1        |         +1        |   011           | X on qubit 3 |
|        +1         |         +1        |         -1        |   100           | X on qubit 4 |
|        -1         |         +1        |         -1        |   101           | X on qubit 5 |
|        +1         |         -1        |         -1        |   110           | X on qubit 6 |
|        -1         |         -1        |         -1        |   111           | X on qubit 7 |

The table for phase-flips is shown below.

| $X_1 X_3 X_5 X_7$ | $X_2 X_3 X_6 X_7$ | $X_4 X_5 X_6 X_7$ | Binary syndrome | Correction   |
| ----------------- | ----------------- | ----------------- | --------------  | ---------    |
|        +1         |         +1        |         +1        |   000           | None         |
|        -1         |         +1        |         +1        |   001           | Z on qubit 1 |
|        +1         |         -1        |         +1        |   010           | Z on qubit 2 |
|        -1         |         -1        |         +1        |   011           | Z on qubit 3 |
|        +1         |         +1        |         -1        |   100           | Z on qubit 4 |
|        -1         |         +1        |         -1        |   101           | Z on qubit 5 |
|        +1         |         -1        |         -1        |   110           | Z on qubit 6 |
|        -1         |         -1        |         -1        |   111           | Z on qubit 7 |

To construct the basis logical states, we will use the following functions:

In [102]:
def steane_logical_zero() -> qld.State:
    amps = [0] * 128
    
    # Superposition of 8 even codeword basis states
    amps[int('0000000', 2)] = 1
    amps[int('1010101', 2)] = 1
    amps[int('0110011', 2)] = 1
    amps[int('1100110', 2)] = 1
    amps[int('0001111', 2)] = 1
    amps[int('1011010', 2)] = 1
    amps[int('0111100', 2)] = 1
    amps[int('0010110', 2)] = 1

    return qld.State(amplitudes=amps)

def steane_logical_one() -> qld.State:
    amps = [0] * 128
    
    # Superposition of 8 odd codeword basis states
    amps[int('1111111', 2)] = 1
    amps[int('0101010', 2)] = 1
    amps[int('1001100', 2)] = 1
    amps[int('0011001', 2)] = 1
    amps[int('1110000', 2)] = 1
    amps[int('0100101', 2)] = 1
    amps[int('1000011', 2)] = 1
    amps[int('1101001', 2)] = 1
    
    return qld.State(amplitudes=amps)

The correction function is defined as follows:

In [100]:
import qlib as ql
import qlib.dense as qld
import numpy as np

def steane_code_correction(state: qld.State):
    if state.num_qubits != 7:
        raise ValueError("Steane code requires 7 qubits")
    
    # Bit-flip stabilizers
    S1 = ql.ops.build_operators(state.num_qubits, [0, 2, 4, 6], [ql.ops.Z, ql.ops.Z, ql.ops.Z, ql.ops.Z])
    S2 = ql.ops.build_operators(state.num_qubits, [1, 2, 5, 6], [ql.ops.Z, ql.ops.Z, ql.ops.Z, ql.ops.Z])
    S3 = ql.ops.build_operators(state.num_qubits, [3, 4, 5, 6], [ql.ops.Z, ql.ops.Z, ql.ops.Z, ql.ops.Z])

    # Phase-flip stabilizers
    S4 = ql.ops.build_operators(state.num_qubits, [0, 2, 4, 6], [ql.ops.X, ql.ops.X, ql.ops.X, ql.ops.X])
    S5 = ql.ops.build_operators(state.num_qubits, [1, 2, 5, 6], [ql.ops.X, ql.ops.X, ql.ops.X, ql.ops.X])
    S6 = ql.ops.build_operators(state.num_qubits, [3, 4, 5, 6], [ql.ops.X, ql.ops.X, ql.ops.X, ql.ops.X])
    
    # Bit-flip measurements
    s1_exp = state.expectation_value(S1)
    s2_exp = state.expectation_value(S2)
    s3_exp = state.expectation_value(S3)

    # Phase-flip measurements
    s4_exp = state.expectation_value(S4)
    s5_exp = state.expectation_value(S5)
    s6_exp = state.expectation_value(S6)

    synd_bitflip = tuple(1 if exp < 0 else 0 for exp in (s1_exp, s2_exp, s3_exp))
    synd_phaseflip = tuple(1 if exp < 0 else 0 for exp in (s4_exp, s5_exp, s6_exp))

    corrections_applied = []

    # Correct X errors
    if any(synd_bitflip):
        error_pos = synd_bitflip[0] + 2 * synd_bitflip[1] + 4 * synd_bitflip[2]
        if error_pos != 0:
            state.apply_gate(ql.ops.X, qubit=(error_pos-1))
            corrections_applied.append(f"X (Q{error_pos-1})")
    
    # Correct Z errors
    if any(synd_phaseflip):
        error_pos = synd_phaseflip[0] + 2 * synd_phaseflip[1] + 4 * synd_phaseflip[2]
        if error_pos > 0:
            state.apply_gate(ql.ops.Z, qubit=(error_pos-1))
            corrections_applied.append(f"Z (Q{error_pos-1})")

    return (s1_exp, s2_exp, s3_exp), synd_bitflip, (s4_exp, s5_exp, s6_exp), synd_phaseflip, " & ".join(corrections_applied)

We will also use a helper function like we did last time:

In [99]:
def run_steane_test(circuit: qld.Circuit):
    state_0 = steane_logical_zero()
    #state_1 = steane_logical_one()
        
    # Create superposition
    #amps_superpos = state_0.matrix.diagonal() + state_1.matrix.diagonal()
    #original_state = ql.State(amplitudes=amps_superpos)
    original_state = state_0
        
    state = circuit.run(initial_state=original_state)
    temp_for_table = []
        
    for i, prob in enumerate(state.probabilities()):
        if prob > 0.001:
            temp_for_table.append(f"P({i:07b}) = {np.abs(prob):.3f}")
    
    states_error = '\n'.join(temp_for_table)
    syndromes = steane_code_correction(state)
    
    temp_for_table.clear()
    for i, prob in enumerate(state.probabilities()):
        if prob > 0.001:
            temp_for_table.append(f"P({i:07b}) = {np.abs(prob):.3f}")
    
    states_corrected = '\n'.join(temp_for_table)

    return syndromes, states_error, states_corrected

The following is a *deterministic error* (noiseless) test of the correction function with a single bit flip. The initial logical state $\ket{\psi_{\mathscr{l}7}}$ is given by:

$$ \ket{\psi_{\mathscr{l}7}} =  \ket{0_{\mathscr{l}7}}. $$

Every bit is flipped one by one, corrected and then the results are documented.

**NOTE**: The reason why it's not a equal superposition is because the table is too big. You'll have to take my word for it that it works, or uncomment the superposition code in the helper function. Also, in the table I've notated the stabilizers as

$$ \begin{align*}
S_1 &= Z_1 Z_3 Z_5 Z_7 \\
S_2 &= Z_2 Z_3 Z_6 Z_7 \\ 
S_3 &= Z_4 Z_5 Z_6 Z_7.
\end{align*}$$

In [103]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(7):
    circuit = qld.Circuit(num_qubits=7)
    circuit.add_gate(ql.ops.X, qubit=i) # Apply a specific X error to qubit i

    (synd, errorstate, correctstate) = run_steane_test(circuit)
    states_error.append(errorstate)
    states_corrected.append(correctstate)
    syndromes.append(synd)

data = {
    "Error": [f"Qubit {i}" for i in range(7)],
    "State after error": states_error,
    "$S_1$": [f"{v[0][0].real:.2f}" for v in syndromes],
    "$S_2$": [f"{v[0][1].real:.2f}" for v in syndromes],
    "$S_3$": [f"{v[0][2].real:.2f}" for v in syndromes],
    "Binary syndrome": [''.join(str(i) for i in v[1][::-1]) for v in syndromes],
    "Correction": [v[4] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>"))

,Error,State after error,$S_1$,$S_2$,$S_3$,Binary syndrome,Correction,State after correction
0,Qubit 0,P(0010101) = 0.125P(0011010) = 0.125P(0100110) = 0.125P(1000000) = 0.125P(1001111) = 0.125P(1010110) = 0.125P(1110011) = 0.125P(1111100) = 0.125,-1.00,1.00,1.00,001,X (Q0),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
1,Qubit 1,P(0010011) = 0.125P(0011100) = 0.125P(0100000) = 0.125P(0101111) = 0.125P(0110110) = 0.125P(1000110) = 0.125P(1110101) = 0.125P(1111010) = 0.125,1.00,-1.00,1.00,010,X (Q1),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
2,Qubit 2,P(0000110) = 0.125P(0010000) = 0.125P(0011111) = 0.125P(0100011) = 0.125P(0101100) = 0.125P(1000101) = 0.125P(1001010) = 0.125P(1110110) = 0.125,-1.00,-1.00,1.00,011,X (Q2),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
3,Qubit 3,P(0000111) = 0.125P(0001000) = 0.125P(0011110) = 0.125P(0110100) = 0.125P(0111011) = 0.125P(1010010) = 0.125P(1011101) = 0.125P(1101110) = 0.125,1.00,1.00,-1.00,100,X (Q3),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
4,Qubit 4,P(0000100) = 0.125P(0001011) = 0.125P(0010010) = 0.125P(0110111) = 0.125P(0111000) = 0.125P(1010001) = 0.125P(1011110) = 0.125P(1100010) = 0.125,-1.00,1.00,-1.00,101,X (Q4),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
5,Qubit 5,P(0000010) = 0.125P(0001101) = 0.125P(0010100) = 0.125P(0110001) = 0.125P(0111110) = 0.125P(1010111) = 0.125P(1011000) = 0.125P(1100100) = 0.125,1.00,-1.00,-1.00,110,X (Q5),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
6,Qubit 6,P(0000001) = 0.125P(0001110) = 0.125P(0010111) = 0.125P(0110010) = 0.125P(0111101) = 0.125P(1010100) = 0.125P(1011011) = 0.125P(1100111) = 0.125,-1.00,-1.00,-1.00,111,X (Q6),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125


The following is a *deterministic error* (noiseless) test of the correction function with a single phase flip. The initial logical state $\ket{\psi_{\mathscr{l}7}}$ is given by:

$$ \ket{\psi_{\mathscr{l}7}} =  \ket{0_{\mathscr{l}7}}. $$

Every bits' phase is flipped one by one, corrected and then the results are documented.

**NOTE**: In the table I've notated the stabilizers as

$$ \begin{align*}
S_1 &= X_1 X_3 X_5 X_7 \\
S_2 &= X_2 X_3 X_6 X_7 \\ 
S_3 &= X_4 X_5 X_6 X_7.
\end{align*}$$

In [104]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(7):
    circuit = qld.Circuit(num_qubits=7)
    circuit.add_gate(ql.ops.Z, qubit=i) # Apply a specific Z error to qubit i

    (synd, errorstate, correctstate) = run_steane_test(circuit)
    states_error.append(errorstate)
    states_corrected.append(correctstate)
    syndromes.append(synd)

data = {
    "Error": [f"Qubit {i}" for i in range(7)],
    "State after error": states_error,
    "$S_1$": [f"{v[2][0].real:.2f}" for v in syndromes],
    "$S_2$": [f"{v[2][1].real:.2f}" for v in syndromes],
    "$S_3$": [f"{v[2][2].real:.2f}" for v in syndromes],
    "Binary syndrome": [''.join(str(i) for i in v[3][::-1]) for v in syndromes],
    "Correction": [v[4] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<div align=\"center\">" + df.to_html().replace("\\n","<br>") + "</div>"))

,Error,State after error,$S_1$,$S_2$,$S_3$,Binary syndrome,Correction,State after correction
0,Qubit 0,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,-0.75,0.75,0.75,001,Z (Q0),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
1,Qubit 1,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,0.75,-0.75,0.75,010,Z (Q1),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
2,Qubit 2,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,-0.75,-0.75,0.75,011,Z (Q2),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
3,Qubit 3,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,0.75,0.75,-0.75,100,Z (Q3),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
4,Qubit 4,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,-0.75,0.75,-0.75,101,Z (Q4),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
5,Qubit 5,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,0.75,-0.75,-0.75,110,Z (Q5),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125
6,Qubit 6,P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125,-0.75,-0.75,-0.75,111,Z (Q6),P(0000000) = 0.125P(0001111) = 0.125P(0010110) = 0.125P(0110011) = 0.125P(0111100) = 0.125P(1010101) = 0.125P(1011010) = 0.125P(1100110) = 0.125


#### 5. Differences between Shor and Steane (Hamming) code

|       Feature      |      Shor Code     |      Steane Code     |
| ------------------ | ------------------ | -------------------- |
| **# Physical qubits**	 |         9          |          7           |
| **Correction capability** |   Corrects any single-qubit error (X, Z, or Y)	| Corrects any single-qubit error (X, Z, or Y)	 |
| **Structure**          | 1st layer: 3-qubit bit-flip code<br>2nd layer: 3-qubit phase-flip code | Uses classical Hamming code structure |
| **# Stabilizers**  |     8           |        6           |
| **Implementation complexity** |  Higher (more qubits, more complex encoding circuit) | Lower (fewer qubits, simpler encoding) |
| **Logical states** |  $\ket{0_{\mathscr{l}9}} = \frac{1}{2 \sqrt{2}} (\ket{000}+\ket{111})^{\otimes 3}$<br>$\ket{1_{\mathscr{l}9}} = \frac{1}{2 \sqrt{2}} (\ket{000}-\ket{111})^{\otimes 3}$  | $\ket{0_{\mathscr{l}7}} = \frac{1}{\sqrt{8}}\sum_{\text{even codewords}}\ket{c}$<br>$\ket{1_{\mathscr{l}7}} = \frac{1}{\sqrt{8}}\sum_{\text{odd codewords}}\ket{c}$ |
| **Error detection** | Can detect more error patterns due to concatenation	 | Efficient but limited to single errors |
| **Practical applications** | Good for demonstrating concatenation principles | More practical for near-term quantum error correction |

#### 6. Challenges along the way

The main challenge was doing the research for Shor and Steane codes. I've touched on repetition code before, but nothing beyond it until now. My main source is a book I have, *Quantum Information* by Stephen M. Barnett; everything else was found with internet searches. The codes are intuitive, especially repetition and Shor codes. The Steane code is intuitive if you understand its classical counterpart, the Hamming code.

Another challenge worth mentioning was putting together a library that allows me to implement the codes and test them—even if its not a particularly good library. All measurements were simulated by taking the *expectation value* of an operator, not actually performing a projective measurement. Nonetheless, it provided valid results which would apply even if projective measurements were used. Shor coding requires 9 physical qubits and 8 ancilla qubits, which is really expensive to simulate given that the library is unoptimized. A 17-qubit system means a $2^{17} = 131072$ dimensional state space, which means a density matrix would hold $131072 \times 131072 = 17.2 \, \text{billion}$ elements. Similarly, Steane code consists of a 13-qubit system (7 physical, 6 ancilla) which means a $2^{13} = 8192$ dimensional space (more manageable than Shor's 131072, but still).

#### 7. Physical implementations
So far we've discussed error correction using expectation values to simulate measurements. This section demonstrates the actual circuit implementations with projective measurements using sparse simulation. So, instead of computing syndrome expectation values, we now use actual ancilla qubits and perform actual measurements, as would be done on real quantum hardware. The following is an implementation of repetition code, as discussed at the end of section 2.

In [183]:
import qlib.sparse as qls
import qlib as ql
from IPython.display import HTML, Latex


circuit = qls.Circuit(num_qubits=5)

# Encoding
circuit.CNOT(control_qubit=0, target_qubit=1)
circuit.CNOT(control_qubit=0, target_qubit=2)

# Noise happens here
# Emulate a fake bitflip
circuit.add_gate(ql.ops.X, qubit=1)
circuit.add_callback(
    lambda state: display(Latex(r"$$\rho_{\text{error}} = " f"{state.to_latex(include_symbol=False)}$$"))
)

# Correction
def correct(state: qls.State, results):
    match results:
        case [1, 0]:
            state.apply_gate(ql.ops.X, qubit=0)
            display(HTML("Corrected qubit 0"))
        case [1, 1]:
            state.apply_gate(ql.ops.X, qubit=1)
            display(HTML("Corrected qubit 1"))
        case [0, 1]:
            state.apply_gate(ql.ops.X, qubit=2)
            display(HTML("Corrected qubit 2"))

circuit.CNOT(control_qubit=0, target_qubit=3)
circuit.CNOT(control_qubit=1, target_qubit=3)
circuit.CNOT(control_qubit=1, target_qubit=4)
circuit.CNOT(control_qubit=2, target_qubit=4)
circuit.measuring_device(qubits=[3, 4], callback=correct)


# Run
state = qls.State(num_qubits=5)
display(state)

final = circuit.run(initial_state=state)
final.latex_symbol = r"\rho_{\text{corrected}}"
display(final)

\rho = |00000\rangle\langle 00000|

<IPython.core.display.Latex object>

\rho_{\text{corrected}} = |00011\rangle\langle 00011|

As you can see, it works. Next is the implementation of the 9-qubit Shor code. I apologize, but there's a lot of code for just setting up the circuit (could probably design the library in a better way to handle stuff like this). I'm too lazy to make a 17-qubit circuit drawing for this, so I'll explain. The first 9 qubits are the data qubits, and the other 8 are ancilla used for syndrome measurement. Encoding is done via CNOTs, similar to repeititon code. The catch is that after we encode the the first qubits of each group, we then switch to the X basis via Hadamard gates, effectively producing the state $\alpha \ket{+++} + \beta \ket{---}$. Only after this do we encode again for bit-flips, which results in the logical states we discussed in section 3.

As mentioned before, a phase-flip in the computational (Z) basis has the effect of a bit-flip in the Hadamard (X) basis. You can see in the circuit, when we perform syndrome measurement for phase-flips we switch back to the computational basis. This allows us to treat the phase-flip as a bit-flip and correct it simply with an $X$ gate on any of the qubits in the error group. After the correction we switch back to the Hadamard basis and perform the real bit-flip checks.

The results show in latex the input state $\rho$ and the output state $\rho^\prime$ after correction. Subsystem $\rho_{\text{data}}$ of the first 9 data qubits is also shown.

In [304]:
import qlib.sparse as qls
import qlib as ql
from IPython.display import HTML, Latex

# 0-8 data bits, 9-16 ancilla qubits for measurement
circuit = qls.Circuit(num_qubits=17)

# Encoding
circuit.CNOT(control_qubit=0, target_qubit=3)
circuit.CNOT(control_qubit=0, target_qubit=6)
circuit.add_gate(ql.ops.H, qubits=[0, 3, 6])

def encode_groups():
    circuit.CNOT(control_qubit=0, target_qubit=1)
    circuit.CNOT(control_qubit=0, target_qubit=2)
    circuit.CNOT(control_qubit=3, target_qubit=4)
    circuit.CNOT(control_qubit=3, target_qubit=5)
    circuit.CNOT(control_qubit=6, target_qubit=7)
    circuit.CNOT(control_qubit=6, target_qubit=8)

def decode_groups():
    circuit.CNOT(control_qubit=0, target_qubit=2)
    circuit.CNOT(control_qubit=0, target_qubit=1)
    circuit.CNOT(control_qubit=3, target_qubit=5)
    circuit.CNOT(control_qubit=3, target_qubit=4)
    circuit.CNOT(control_qubit=6, target_qubit=8)
    circuit.CNOT(control_qubit=6, target_qubit=7)

encode_groups()

# Noise happens here (phase flip in group 2, and a bit-flip in each group)
circuit.add_gate(ql.ops.Z, qubit=3)
circuit.add_gate(ql.ops.X, qubit=0)
circuit.add_gate(ql.ops.X, qubit=5)
circuit.add_gate(ql.ops.X, qubit=7)

# Correction
def correct_phase(state: qls.State, results):
    match results:
        case [1, 0]:
            state.apply_gate(ql.ops.X, qubit=0)
            display(HTML("Corrected phase on G0"))
        case [1, 1]:
            state.apply_gate(ql.ops.X, qubit=3)
            display(HTML("Corrected phase on G1"))
        case [0, 1]:
            state.apply_gate(ql.ops.X, qubit=6)
            display(HTML("Corrected phase on G2"))

# Decode inner code (bit-flip)
decode_groups()
# Back to computational basis
circuit.add_gate(ql.ops.H, qubits=[0, 3, 6])
circuit.CNOT(control_qubit=0, target_qubit=9)
circuit.CNOT(control_qubit=3, target_qubit=9)
circuit.CNOT(control_qubit=3, target_qubit=10)
circuit.CNOT(control_qubit=6, target_qubit=10)
circuit.measuring_device(qubits=[9, 10], callback=correct_phase) # Measure and correct
circuit.add_gate(ql.ops.H, qubits=[0, 3, 6]) # Back to Hadamard basis
# Encode again
encode_groups()

def correct_bitflip(state: qls.State, results, group):
    match results:
        case [1, 0]:
            state.apply_gate(ql.ops.X, qubit=group*3)
            display(HTML(f"Corrected bit-flip on G{group} Q{group*3}"))
        case [1, 1]:
            state.apply_gate(ql.ops.X, qubit=group*3 + 1)
            display(HTML(f"Corrected bit-flip on G{group} Q{group*3 + 1}"))
        case [0, 1]:
            state.apply_gate(ql.ops.X, qubit=group*3 + 2)
            display(HTML(f"Corrected bit-flip on G{group} Q{group*3 + 2}"))

def group_correction(group):
    circuit.CNOT(control_qubit=group*3, target_qubit=group*2 + 11)
    circuit.CNOT(control_qubit=group*3 + 1, target_qubit=group*2 + 11)
    circuit.CNOT(control_qubit=group*3 + 1, target_qubit=group*2 + 12)
    circuit.CNOT(control_qubit=group*3 + 2, target_qubit=group*2 + 12)
    circuit.measuring_device(qubits=[group*2 + 11, group*2 + 12], callback=lambda state, res: correct_bitflip(state, res, group))

group_correction(0)
group_correction(1)
group_correction(2)


# Decode the state so we can compare it to the input state
decode_groups()
circuit.add_gate(ql.ops.H, qubits=[0, 3, 6])
circuit.CNOT(control_qubit=0, target_qubit=5)
circuit.CNOT(control_qubit=0, target_qubit=3)


# Finally, run the fucking circuit...
state = qls.State(num_qubits=17)
final = circuit.run(initial_state=state)
final.latex_symbol = r"\rho^{\prime}"

sub_state = state.partial_trace(keep_qubits=range(9))
sub_state.latex_symbol = r"\rho_{\text{data}}"
sub_final = final.partial_trace(keep_qubits=range(9))
sub_final.latex_symbol = r"\rho^{\prime}_{\text{data}}"

display(state, final)
display(sub_state, sub_final)

\rho = |00000000000000000\rangle\langle 00000000000000000|

\rho^{\prime} = |00000000011100111\rangle\langle 00000000011100111|

\rho_{\text{data}} = |000000000\rangle\langle 000000000|

\rho^{\prime}_{\text{data}} = |000000000\rangle\langle 000000000|